### 1.装饰器实现单实例模式

In [22]:
def singleton(cls):
    instance = {}
    def getinstance():
        if cls not in instance:
            instance[cls] = cls()
        print(instance)
        return instance[cls]
    return getinstance

@singleton
class MyClass1(object):
    pass

@singleton
class MyClass2(object):
    pass

* 验证def singleton() 中的instance的可见范围
* MyClass1 、MyClass2 有各自的instance；
* m3创建时，回到MyClass1的可见范围里。

In [27]:
m1 = MyClass1()

{<class '__main__.MyClass1'>: <__main__.MyClass1 object at 0x1031a1750>}


In [28]:
m2 = MyClass2()

{<class '__main__.MyClass2'>: <__main__.MyClass2 object at 0x103161f10>}


In [29]:
m3 = MyClass1()

{<class '__main__.MyClass1'>: <__main__.MyClass1 object at 0x1031a1750>}


### `__new__` 与 `__init__` 的关系

In [89]:
class Foo(object):
    def __new__(cls, name):
        print(f'Trace __new__({cls})')
        return super().__new__(cls)
    
    def __init__(self, name):
        super().__init__()
        print(f'Trace __init__({self})')
        self.name = name

In [90]:
bar = Foo('test')

Trace __new__(<class '__main__.Foo'>)
Trace __init__(<__main__.Foo object at 0x1031b0e10>)


In [95]:
#相对于下面，分开操作：
bar = Foo.__new__(Foo, 'test')
bar

Trace __new__(<class '__main__.Foo'>)


In [96]:
if isinstance(bar, Foo):
    Foo.__init__(bar, 'abc')

Trace __init__(<__main__.Foo object at 0x1031d58d0>)


In [97]:
bar.nae

'abc'

### 2.用`__new()__`实现单实例模式：

In [107]:
class Singleton2(object):
    _isinstance = False
    def __new__(cls):
        if cls._isinstance:
            return cls._isinstance
        cls._isinstance = super().__new__(cls)
        return cls._isinstance

In [108]:
s1 = Singleton2()
s2 = Singleton2()
id(s1), id(s2)

(4347047056, 4347047056)

## 解决并发，引入带锁版

In [135]:
import threading

In [142]:
class Singleton3(object):
    objs = {}
    objs_locker = threading.Lock()
    def __new__(cls, *args, **kwargs):
        if cls in cls.objs:
            return cls.objs[cls]
        cls.objs_locker.acquire()
        try:
            if cls in cls.objs:  ##duble checking
                return cls.objs[cls]
            cls.objs[cls] = object.__new__(cls)
        finally:
            cls.objs_locker.release()

In [143]:
s1 = Singleton3()
s2 = Singleton3()

In [144]:
id(s1), id(s2)

(4304996456, 4390082896)

In [145]:
import threading
class Singleton(object):
    objs = {}
    objs_locker = threading.Lock()
    def __new__(cls, *args, **kargs):
        if cls in cls.objs:
            return cls.objs[cls]
        cls.objs_locker.acquire()
        try:
            if cls in cls.objs: ## double check locking
                return cls.objs[cls]
            cls.objs[cls] = object.__new__(cls)
        finally:
            cls.objs_locker.release()

In [146]:
s1 = Singleton()
s2 = Singleton()

In [147]:
id(s1), id(s2)

(4304996456, 4414431504)